In [ ]:
#Required Libraries
import os
import pandas as pd
from SpatioTemporal_calculation import compute_spatiotemporal_variables
from gait_events import gait_events_HC_JA
import importlib
import gait_events
importlib.reload(gait_events)
sampling_rate = 200 
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product

In [2]:
#Function to load a patient's data ---- inputs patient_id & group_code
def load_patient_data(patient_folder, patient_id, group_code, verbose=False):
    df_list = []
    file_list = []
    
    d_values = ['D01', 'D02']
    b_values = ['B01', 'B02', 'B03']
    t_values = ['T01', 'T02', 'T03']
    
    for d, b, t in product(d_values, b_values, t_values):
        file_name = os.path.join(patient_folder, f"{patient_id}_{group_code}_{d}_{b}_{t}.csv")
        if os.path.exists(file_name):
            try:
                # Read CSV with reduced memory usage (example dtypes)
                df = pd.read_csv(file_name, dtype=float)  # adjust dtypes based on your data
                if df.empty:
                    if verbose:
                        print(f"Empty file skipped: {file_name}")
                    continue
                
                # Add metadata columns
                df['patient_id'] = patient_id
                df['group'] = group_code
                df['day'] = d
                df['block'] = b
                df['trial'] = t
                
                df_list.append(df)
                file_list.append(file_name)
            except Exception as e:
                if verbose:
                    print(f"Error reading {file_name}: {e}")
        else:
            if verbose:
                print(f"File not found: {file_name}")
    
    if df_list:
        patient_df = pd.concat(df_list, ignore_index=True)
        return patient_df
    else:
        return pd.DataFrame()

In [3]:
# Function to process and save patient data ---- inputs patient_df, output_folder, patient_id
def process_and_save_patient(patient_df, output_folder, patient_id):
    """
    Process patient data and save a summary file to avoid re-loading everything.
    """
    if patient_df.empty:
        return
    
    # Example: calculate basic stats by column
    summary = patient_df.describe().transpose()
    
    # Save summary as CSV or pickle
    summary_path = os.path.join(output_folder, f"{patient_id}_summary.csv")
    summary.to_csv(summary_path)
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    print(f"Saved summary for patient {patient_id} at {summary_path}")

In [4]:
#Function to load group data --- inputs base_folder, group_code, output_folder
def load_group_data(base_folder, group_code, output_folder, verbose=False):
    patient_ids = sorted([folder for folder in os.listdir(base_folder) if folder.startswith("S") and len(folder) == 4])
    
    for patient_id in patient_ids:
        patient_folder = os.path.join(base_folder, patient_id)
        patient_df = load_patient_data(patient_folder, patient_id, group_code, verbose)
        
        if verbose:
            print(f"Processing patient {patient_id}, data shape: {patient_df.shape}")
        
        process_and_save_patient(patient_df, output_folder, patient_id)

In [8]:
if __name__ == "__main__":
    base_folder_g1 = "./young adults (19–35 years old)"
    base_folder_g2 = "./old adults (56+ years old)"
    output_folder_g1 = "./EDA/G1_summaries"
    if not os.path.exists(output_folder_g1):
        os.makedirs(output_folder_g1)
    output_folder_g2 = "./EDA/G2_summaries"
    if not os.path.exists(output_folder_g2):
        os.makedirs(output_folder_g2)
    
    print("Processing G1 patients...")
    load_group_data(base_folder_g1, "G01", output_folder_g1, verbose=True)
    
    print("\nProcessing G2 patients...")
    load_group_data(base_folder_g2, "G02", output_folder_g2, verbose=True)

Processing G1 patients...
Processing patient S002, data shape: (864018, 326)
Saved summary for patient S002 at ./EDA/G1_summaries/S002_summary.csv
Processing patient S003, data shape: (864018, 326)
Saved summary for patient S003 at ./EDA/G1_summaries/S003_summary.csv
Processing patient S004, data shape: (864018, 326)
Saved summary for patient S004 at ./EDA/G1_summaries/S004_summary.csv
Processing patient S005, data shape: (864018, 326)
Saved summary for patient S005 at ./EDA/G1_summaries/S005_summary.csv
Processing patient S006, data shape: (864018, 326)
Saved summary for patient S006 at ./EDA/G1_summaries/S006_summary.csv
Processing patient S007, data shape: (864018, 326)
Saved summary for patient S007 at ./EDA/G1_summaries/S007_summary.csv
Error reading ./young adults (19–35 years old)/S008/S008_G01_D01_B03_T02.csv: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.


: 

: 

In [ ]:
# Young adults (19–35 years old) Mean values 
def load_patient_data(patient_folder, patient_id):
    df_list = []
    file_list = []
    
    for d in ['D01', 'D02']:
        for b in ['B01', 'B02', 'B03']:
            for t in ['T01', 'T02', 'T03']:
                # Aquí se usa el patient_id para construir el nombre del archivo
                file_name = f"{patient_folder}/{patient_id}_G01_{d}_{b}_{t}.csv"
                if os.path.exists(file_name):
                    try:
                        df = pd.read_csv(file_name)
                        if df.empty:
                            print(f"Archivo vacío: {file_name}")
                            continue
                        df_list.append(df)
                        file_list.append(file_name)
                    except Exception as e:
                        print(f"Error al leer {file_name}: {e}")
                else:
                    print(f"Archivo no encontrado: {file_name}")
    return df_list, file_list

# Define la carpeta base donde se encuentran los pacientes
base_folder = "./young adults (19–35 years old)"
output_folder = "./EDA/Mean"

# Verifica que la carpeta EDA existe, si no la crea
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Lista para almacenar los resultados de todos los pacientes
all_results = []

# Obtener los IDs de los pacientes de la carpeta base (solo IDs únicos)
patient_ids = sorted(set(folder for folder in os.listdir(base_folder) if folder.startswith("S") and len(folder) == 4))

# Verifica que se obtuvieron los IDs de los pacientes correctamente
print(f"Lista de pacientes: {patient_ids}")

for patient_id in patient_ids:
    patient_folder = os.path.join(base_folder, patient_id)

    # Cargar todos los DataFrames y nombres de archivo para este paciente
    df_list, file_list = load_patient_data(patient_folder, patient_id)

    # Lista para almacenar los resultados de cada toma
    results = []

    for i, df in enumerate(df_list):
        trial_name = file_list[i]
        print(f"\nProcesando la toma: {trial_name}")
        
        # Paso 1: Detección de eventos de marcha
        try:
            print("  Ejecutando la detección de gait events...")
            heel_strike_R, heel_strike_L, toe_off_R, toe_off_L = gait_events_HC_JA(df)
            print(f"  Detección de gait events exitosa para {trial_name}:")
        except Exception as e:
            print(f"  Error en la detección de gait events para {trial_name}: {e}")
            continue  # Salta esta toma si falla la detección de eventos

        # Verifica si se detectaron suficientes eventos
        if not heel_strike_R or not toe_off_R:
            print(f"  Se omite la toma {trial_name} por eventos insuficientes.")
            continue

        # Paso 2: Cálculo de variables espaciotemporales
        try:
            print("  Ejecutando el cálculo de variables espaciotemporales...")
            spatiotemporal_df, mean_df, std_df = compute_spatiotemporal_variables(df, heel_strike_R, heel_strike_L, toe_off_R, toe_off_L, sampling_rate)
            if mean_df.empty:
                print(f"  Se omite la toma {trial_name} por resultados espaciotemporales vacíos.")
                continue
            print(f"  Cálculo de variables espaciotemporales completado para {trial_name}.")
        except Exception as e:
            print(f"  Error en el cálculo espaciotemporal para {trial_name}: {e}")
            continue

        # Agrega una columna para identificar la toma
        mean_df['Trial'] = os.path.basename(trial_name)
        results.append(mean_df)

    # Combina los resultados de todas las tomas en un solo DataFrame
    if results:
        final_df = pd.concat(results, ignore_index=True)
        print("\nVista previa del DataFrame final:")
        print(final_df.head())

        # Ruta del archivo de salida en la carpeta EDA
        output_file = os.path.join(output_folder, f"spatiotemporal_G1_{patient_id}.csv")

        # Elimina el archivo existente si ya existe
        if os.path.exists(output_file):
            os.remove(output_file)
            print(f"Archivo existente eliminado: {output_file}")

        # Guardar el DataFrame final en un archivo CSV con el patient_id en la carpeta EDA
        final_df.to_csv(output_file, index=False)
    else:
        print(f"No se generaron resultados para el paciente {patient_id}.")


Lista de pacientes: ['S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S010', 'S012', 'S014', 'S015', 'S016', 'S017', 'S018', 'S020', 'S021', 'S023', 'S024', 'S025', 'S026', 'S027', 'S028', 'S030', 'S031', 'S032', 'S033', 'S034', 'S035', 'S036', 'S037', 'S038', 'S039', 'S063']

Procesando la toma: ./young adults (19–35 years old)/S002/S002_G01_D01_B01_T01.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./young adults (19–35 years old)/S002/S002_G01_D01_B01_T01.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./young adults (19–35 years old)/S002/S002_G01_D01_B01_T01.csv.

Procesando la toma: ./young adults (19–35 years old)/S002/S002_G01_D01_B01_T02.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./young adults (19–35 years old)/S002/S002_G01_D01_B01_T02.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de v

/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:369: RuntimeWarning: Mean of empty slice
  "Right Step Width (cm)": np.nanmean(right_step_width),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:370: RuntimeWarning: Mean of empty slice
  "Left Step Width (cm)": np.nanmean(left_step_width),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:371: RuntimeWarning: Mean of empty slice
  "Right Stride Length (cm)": np.nanmean(right_stride_length),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:372: RuntimeWarning: Mean of empty slice
  "Left Stride Length (cm)": np.nanmean(left_stride_length),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:373: RuntimeWarning: Mean of empty slice
  "Right Step Length (cm)": np.nanmean(right_step_length),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:374: RuntimeWarning: Mean of empty slice
  "Left Step Length (cm)": np.nanmean(left_step_length),
/home/

  Cálculo de variables espaciotemporales completado para ./young adults (19–35 years old)/S025/S025_G01_D01_B01_T02.csv.

Procesando la toma: ./young adults (19–35 years old)/S025/S025_G01_D01_B01_T03.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./young adults (19–35 years old)/S025/S025_G01_D01_B01_T03.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./young adults (19–35 years old)/S025/S025_G01_D01_B01_T03.csv.

Procesando la toma: ./young adults (19–35 years old)/S025/S025_G01_D01_B02_T01.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./young adults (19–35 years old)/S025/S025_G01_D01_B02_T01.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./young adults (19–35 years old)/S025/S025_G01_D01_B02_T01.csv.

Procesando la toma: ./young adults (19–35 years old)/S02

/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:383: RuntimeWarning: Mean of empty slice
  "Double Support Time (s)": np.nanmean(double_support_time),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:384: RuntimeWarning: Mean of empty slice
  "Single Support Time (s)": np.nanmean(single_support_time),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:385: RuntimeWarning: Mean of empty slice
  "Percentage Double Support (%)": np.nanmean(pct_double),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:386: RuntimeWarning: Mean of empty slice
  "Percentage Single Support (%)": np.nanmean(pct_single),
/home/dmartinez/miniconda3/envs/inv_Di/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


  Cálculo de variables espaciotemporales completado para ./young adults (19–35 years old)/S028/S028_G01_D01_B03_T01.csv.

Procesando la toma: ./young adults (19–35 years old)/S028/S028_G01_D01_B03_T02.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./young adults (19–35 years old)/S028/S028_G01_D01_B03_T02.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./young adults (19–35 years old)/S028/S028_G01_D01_B03_T02.csv.

Procesando la toma: ./young adults (19–35 years old)/S028/S028_G01_D01_B03_T03.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./young adults (19–35 years old)/S028/S028_G01_D01_B03_T03.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./young adults (19–35 years old)/S028/S028_G01_D01_B03_T03.csv.

Procesando la toma: ./young adults (19–35 years old)/S02

/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:383: RuntimeWarning: Mean of empty slice
  "Double Support Time (s)": np.nanmean(double_support_time),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:384: RuntimeWarning: Mean of empty slice
  "Single Support Time (s)": np.nanmean(single_support_time),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:385: RuntimeWarning: Mean of empty slice
  "Percentage Double Support (%)": np.nanmean(pct_double),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:386: RuntimeWarning: Mean of empty slice
  "Percentage Single Support (%)": np.nanmean(pct_single),
/home/dmartinez/miniconda3/envs/inv_Di/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


  Cálculo de variables espaciotemporales completado para ./young adults (19–35 years old)/S039/S039_G01_D01_B01_T03.csv.

Procesando la toma: ./young adults (19–35 years old)/S039/S039_G01_D01_B02_T01.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./young adults (19–35 years old)/S039/S039_G01_D01_B02_T01.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./young adults (19–35 years old)/S039/S039_G01_D01_B02_T01.csv.

Procesando la toma: ./young adults (19–35 years old)/S039/S039_G01_D01_B02_T02.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./young adults (19–35 years old)/S039/S039_G01_D01_B02_T02.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./young adults (19–35 years old)/S039/S039_G01_D01_B02_T02.csv.

Procesando la toma: ./young adults (19–35 years old)/S03

In [ ]:
# Old adults (56+ years old) Mean values
def load_patient_data(patient_folder, patient_id):
    df_list = []
    file_list = []
    
    for d in ['D01', 'D02']:
        for b in ['B01', 'B02', 'B03']:
            for t in ['T01', 'T02', 'T03']:
                # Aquí se usa el patient_id para construir el nombre del archivo
                file_name = f"{patient_folder}/{patient_id}_G03_{d}_{b}_{t}.csv"
                if os.path.exists(file_name):
                    try:
                        df = pd.read_csv(file_name)
                        if df.empty:
                            print(f"Archivo vacío: {file_name}")
                            continue
                        df_list.append(df)
                        file_list.append(file_name)
                    except Exception as e:
                        print(f"Error al leer {file_name}: {e}")
                else:
                    print(f"Archivo no encontrado: {file_name}")
    return df_list, file_list

# Define la carpeta base donde se encuentran los pacientes
base_folder = "./old adults (56+ years old)"
output_folder = "./EDA/Mean"

# Verifica que la carpeta EDA existe, si no la crea
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Lista para almacenar los resultados de todos los pacientes
all_results = []

# Obtener los IDs de los pacientes de la carpeta base (solo IDs únicos)
patient_ids = sorted(set(folder for folder in os.listdir(base_folder) if folder.startswith("S") and len(folder) == 4))

# Verifica que se obtuvieron los IDs de los pacientes correctamente
print(f"Lista de pacientes: {patient_ids}")

for patient_id in patient_ids:
    patient_folder = os.path.join(base_folder, patient_id)

    # Cargar todos los DataFrames y nombres de archivo para este paciente
    df_list, file_list = load_patient_data(patient_folder, patient_id)

    # Lista para almacenar los resultados de cada toma
    results = []

    for i, df in enumerate(df_list):
        trial_name = file_list[i]
        print(f"\nProcesando la toma: {trial_name}")
        
        # Paso 1: Detección de eventos de marcha
        try:
            print("  Ejecutando la detección de gait events...")
            heel_strike_R, heel_strike_L, toe_off_R, toe_off_L = gait_events_HC_JA(df)
            print(f"  Detección de gait events exitosa para {trial_name}:")
        except Exception as e:
            print(f"  Error en la detección de gait events para {trial_name}: {e}")
            continue  # Salta esta toma si falla la detección de eventos

        # Verifica si se detectaron suficientes eventos
        if not heel_strike_R or not toe_off_R:
            print(f"  Se omite la toma {trial_name} por eventos insuficientes.")
            continue

        # Paso 2: Cálculo de variables espaciotemporales
        try:
            print("  Ejecutando el cálculo de variables espaciotemporales...")
            spatiotemporal_df, mean_df, std_df = compute_spatiotemporal_variables(df, heel_strike_R, heel_strike_L, toe_off_R, toe_off_L, sampling_rate)
            if mean_df.empty:
                print(f"  Se omite la toma {trial_name} por resultados espaciotemporales vacíos.")
                continue
            print(f"  Cálculo de variables espaciotemporales completado para {trial_name}.")
        except Exception as e:
            print(f"  Error en el cálculo espaciotemporal para {trial_name}: {e}")
            continue

        # Agrega una columna para identificar la toma
        mean_df['Trial'] = os.path.basename(trial_name)
        results.append(mean_df)

    # Combina los resultados de todas las tomas en un solo DataFrame
    if results:
        final_df = pd.concat(results, ignore_index=True)
        print("\nVista previa del DataFrame final:")
        print(final_df.head())

        # Ruta del archivo de salida en la carpeta EDA
        output_file = os.path.join(output_folder, f"spatiotemporal_G3_{patient_id}.csv")

        # Elimina el archivo existente si ya existe
        if os.path.exists(output_file):
            os.remove(output_file)
            print(f"Archivo existente eliminado: {output_file}")

        # Guardar el DataFrame final en un archivo CSV con el patient_id en la carpeta EDA
        final_df.to_csv(output_file, index=False)
    else:
        print(f"No se generaron resultados para el paciente {patient_id}.")


Lista de pacientes: ['S009', 'S013', 'S019', 'S040', 'S081', 'S102', 'S103', 'S104', 'S107', 'S109', 'S110', 'S112', 'S113', 'S116', 'S119', 'S120', 'S123', 'S125', 'S126', 'S127', 'S128', 'S129', 'S130', 'S138', 'S139', 'S140', 'S141', 'S142', 'S143', 'S145', 'S146', 'S154', 'S156', 'S157', 'S158']

Procesando la toma: ./old adults (56+ years old)/S009/S009_G03_D01_B01_T01.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./old adults (56+ years old)/S009/S009_G03_D01_B01_T01.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./old adults (56+ years old)/S009/S009_G03_D01_B01_T01.csv.

Procesando la toma: ./old adults (56+ years old)/S009/S009_G03_D01_B01_T02.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./old adults (56+ years old)/S009/S009_G03_D01_B01_T02.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de varia

/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:212: RuntimeWarning: invalid value encountered in sqrt
  area = np.sqrt(s * (s - dist_L_to_R) * (s - dist_nextL_to_R) * (s - c))


  Cálculo de variables espaciotemporales completado para ./old adults (56+ years old)/S110/S110_G03_D01_B01_T02.csv.

Procesando la toma: ./old adults (56+ years old)/S110/S110_G03_D01_B01_T03.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./old adults (56+ years old)/S110/S110_G03_D01_B01_T03.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./old adults (56+ years old)/S110/S110_G03_D01_B01_T03.csv.

Procesando la toma: ./old adults (56+ years old)/S110/S110_G03_D01_B02_T01.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./old adults (56+ years old)/S110/S110_G03_D01_B02_T01.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./old adults (56+ years old)/S110/S110_G03_D01_B02_T01.csv.

Procesando la toma: ./old adults (56+ years old)/S110/S110_G03_D01_B02_T02.csv
  Eje

/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:383: RuntimeWarning: Mean of empty slice
  "Double Support Time (s)": np.nanmean(double_support_time),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:384: RuntimeWarning: Mean of empty slice
  "Single Support Time (s)": np.nanmean(single_support_time),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:385: RuntimeWarning: Mean of empty slice
  "Percentage Double Support (%)": np.nanmean(pct_double),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:386: RuntimeWarning: Mean of empty slice
  "Percentage Single Support (%)": np.nanmean(pct_single),
/home/dmartinez/miniconda3/envs/inv_Di/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


  Cálculo de variables espaciotemporales completado para ./old adults (56+ years old)/S126/S126_G03_D01_B02_T03.csv.

Procesando la toma: ./old adults (56+ years old)/S126/S126_G03_D01_B03_T01.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./old adults (56+ years old)/S126/S126_G03_D01_B03_T01.csv:
  Ejecutando el cálculo de variables espaciotemporales...


/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:383: RuntimeWarning: Mean of empty slice
  "Double Support Time (s)": np.nanmean(double_support_time),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:384: RuntimeWarning: Mean of empty slice
  "Single Support Time (s)": np.nanmean(single_support_time),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:385: RuntimeWarning: Mean of empty slice
  "Percentage Double Support (%)": np.nanmean(pct_double),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:386: RuntimeWarning: Mean of empty slice
  "Percentage Single Support (%)": np.nanmean(pct_single),
/home/dmartinez/miniconda3/envs/inv_Di/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


  Cálculo de variables espaciotemporales completado para ./old adults (56+ years old)/S126/S126_G03_D01_B03_T01.csv.

Procesando la toma: ./old adults (56+ years old)/S126/S126_G03_D01_B03_T02.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./old adults (56+ years old)/S126/S126_G03_D01_B03_T02.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./old adults (56+ years old)/S126/S126_G03_D01_B03_T02.csv.

Procesando la toma: ./old adults (56+ years old)/S126/S126_G03_D01_B03_T03.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./old adults (56+ years old)/S126/S126_G03_D01_B03_T03.csv:
  Se omite la toma ./old adults (56+ years old)/S126/S126_G03_D01_B03_T03.csv por eventos insuficientes.

Vista previa del DataFrame final:
   Right Step Width (cm)  Left Step Width (cm)  Right Stride Length (cm)  \
0              11.232671              8.4190

In [2]:
# Young adults (19–35 years old) STD values 
def load_patient_data(patient_folder, patient_id):
    df_list = []
    file_list = []
    
    for d in ['D01', 'D02']:
        for b in ['B01', 'B02', 'B03']:
            for t in ['T01', 'T02', 'T03']:
                # Aquí se usa el patient_id para construir el nombre del archivo
                file_name = f"{patient_folder}/{patient_id}_G01_{d}_{b}_{t}.csv"
                if os.path.exists(file_name):
                    try:
                        df = pd.read_csv(file_name)
                        if df.empty:
                            print(f"Archivo vacío: {file_name}")
                            continue
                        df_list.append(df)
                        file_list.append(file_name)
                    except Exception as e:
                        print(f"Error al leer {file_name}: {e}")
                else:
                    print(f"Archivo no encontrado: {file_name}")
    return df_list, file_list

# Define la carpeta base donde se encuentran los pacientes
base_folder = "./young adults (19–35 years old)"
output_folder = "./EDA/STD"

# Verifica que la carpeta EDA existe, si no la crea
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Lista para almacenar los resultados de todos los pacientes
all_results = []

# Obtener los IDs de los pacientes de la carpeta base (solo IDs únicos)
patient_ids = sorted(set(folder for folder in os.listdir(base_folder) if folder.startswith("S") and len(folder) == 4))

# Verifica que se obtuvieron los IDs de los pacientes correctamente
print(f"Lista de pacientes: {patient_ids}")

for patient_id in patient_ids:
    patient_folder = os.path.join(base_folder, patient_id)

    # Verificar si ya se ha procesado este paciente
    output_file = os.path.join(output_folder, f"spatiotemporal_G1_{patient_id}.csv")
    if os.path.exists(output_file):
        print(f"El paciente {patient_id} ya ha sido procesado. Se omite.")
        continue  # Omite este paciente y pasa al siguiente

    # Cargar todos los DataFrames y nombres de archivo para este paciente
    df_list, file_list = load_patient_data(patient_folder, patient_id)

    # Lista para almacenar los resultados de cada toma
    results = []

    for i, df in enumerate(df_list):
        trial_name = file_list[i]
        print(f"\nProcesando la toma: {trial_name}")
        
        # Paso 1: Detección de eventos de marcha
        try:
            print("  Ejecutando la detección de gait events...")
            heel_strike_R, heel_strike_L, toe_off_R, toe_off_L = gait_events_HC_JA(df)
            print(f"  Detección de gait events exitosa para {trial_name}:")
        except Exception as e:
            print(f"  Error en la detección de gait events para {trial_name}: {e}")
            continue  # Salta esta toma si falla la detección de eventos

        # Verifica si se detectaron suficientes eventos
        if not heel_strike_R or not toe_off_R:
            print(f"  Se omite la toma {trial_name} por eventos insuficientes.")
            continue

        # Paso 2: Cálculo de variables espaciotemporales
        try:
            print("  Ejecutando el cálculo de variables espaciotemporales...")
            spatiotemporal_df, mean_df, std_df = compute_spatiotemporal_variables(df, heel_strike_R, heel_strike_L, toe_off_R, toe_off_L, sampling_rate)
            if std_df.empty:
                print(f"  Se omite la toma {trial_name} por resultados espaciotemporales vacíos.")
                continue
            print(f"  Cálculo de variables espaciotemporales completado para {trial_name}.")
        except Exception as e:
            print(f"  Error en el cálculo espaciotemporal para {trial_name}: {e}")
            continue

        # Agrega una columna para identificar la toma
        std_df['Trial'] = os.path.basename(trial_name)
        results.append(std_df)

    # Combina los resultados de todas las tomas en un solo DataFrame
    if results:
        final_df = pd.concat(results, ignore_index=True)
        print("\nVista previa del DataFrame final:")
        print(final_df.head())

        # Ruta del archivo de salida en la carpeta EDA
        output_file = os.path.join(output_folder, f"spatiotemporal_G1_{patient_id}.csv")

        # Elimina el archivo existente si ya existe
        if os.path.exists(output_file):
            os.remove(output_file)
            print(f"Archivo existente eliminado: {output_file}")

        # Guardar el DataFrame final en un archivo CSV con el patient_id en la carpeta EDA
        final_df.to_csv(output_file, index=False)
    else:
        print(f"No se generaron resultados para el paciente {patient_id}.")

Lista de pacientes: ['S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S010', 'S012', 'S014', 'S015', 'S016', 'S017', 'S018', 'S020', 'S021', 'S023', 'S024', 'S025', 'S026', 'S027', 'S028', 'S030', 'S031', 'S032', 'S033', 'S034', 'S035', 'S036', 'S037', 'S038', 'S039', 'S063']
El paciente S002 ya ha sido procesado. Se omite.
El paciente S003 ya ha sido procesado. Se omite.
El paciente S004 ya ha sido procesado. Se omite.
El paciente S005 ya ha sido procesado. Se omite.
El paciente S006 ya ha sido procesado. Se omite.
El paciente S007 ya ha sido procesado. Se omite.
El paciente S008 ya ha sido procesado. Se omite.
El paciente S010 ya ha sido procesado. Se omite.
El paciente S012 ya ha sido procesado. Se omite.
El paciente S014 ya ha sido procesado. Se omite.
El paciente S015 ya ha sido procesado. Se omite.
El paciente S016 ya ha sido procesado. Se omite.
El paciente S017 ya ha sido procesado. Se omite.
El paciente S018 ya ha sido procesado. Se omite.
El paciente S020 ya ha sido p

/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:383: RuntimeWarning: Mean of empty slice
  "Double Support Time (s)": np.nanmean(double_support_time),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:384: RuntimeWarning: Mean of empty slice
  "Single Support Time (s)": np.nanmean(single_support_time),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:385: RuntimeWarning: Mean of empty slice
  "Percentage Double Support (%)": np.nanmean(pct_double),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:386: RuntimeWarning: Mean of empty slice
  "Percentage Single Support (%)": np.nanmean(pct_single),
/home/dmartinez/miniconda3/envs/inv_Di/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


  Cálculo de variables espaciotemporales completado para ./young adults (19–35 years old)/S028/S028_G01_D01_B03_T01.csv.

Procesando la toma: ./young adults (19–35 years old)/S028/S028_G01_D01_B03_T02.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./young adults (19–35 years old)/S028/S028_G01_D01_B03_T02.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./young adults (19–35 years old)/S028/S028_G01_D01_B03_T02.csv.

Procesando la toma: ./young adults (19–35 years old)/S028/S028_G01_D01_B03_T03.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./young adults (19–35 years old)/S028/S028_G01_D01_B03_T03.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./young adults (19–35 years old)/S028/S028_G01_D01_B03_T03.csv.

Procesando la toma: ./young adults (19–35 years old)/S02

/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:383: RuntimeWarning: Mean of empty slice
  "Double Support Time (s)": np.nanmean(double_support_time),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:384: RuntimeWarning: Mean of empty slice
  "Single Support Time (s)": np.nanmean(single_support_time),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:385: RuntimeWarning: Mean of empty slice
  "Percentage Double Support (%)": np.nanmean(pct_double),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:386: RuntimeWarning: Mean of empty slice
  "Percentage Single Support (%)": np.nanmean(pct_single),
/home/dmartinez/miniconda3/envs/inv_Di/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


  Cálculo de variables espaciotemporales completado para ./young adults (19–35 years old)/S039/S039_G01_D01_B01_T03.csv.

Procesando la toma: ./young adults (19–35 years old)/S039/S039_G01_D01_B02_T01.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./young adults (19–35 years old)/S039/S039_G01_D01_B02_T01.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./young adults (19–35 years old)/S039/S039_G01_D01_B02_T01.csv.

Procesando la toma: ./young adults (19–35 years old)/S039/S039_G01_D01_B02_T02.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./young adults (19–35 years old)/S039/S039_G01_D01_B02_T02.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./young adults (19–35 years old)/S039/S039_G01_D01_B02_T02.csv.

Procesando la toma: ./young adults (19–35 years old)/S03

In [4]:
# Old adults (56+ years old) STD values 
def load_patient_data(patient_folder, patient_id):
    df_list = []
    file_list = []
    
    for d in ['D01', 'D02']:
        for b in ['B01', 'B02', 'B03']:
            for t in ['T01', 'T02', 'T03']:
                # Aquí se usa el patient_id para construir el nombre del archivo
                file_name = f"{patient_folder}/{patient_id}_G03_{d}_{b}_{t}.csv"
                if os.path.exists(file_name):
                    try:
                        df = pd.read_csv(file_name)
                        if df.empty:
                            print(f"Archivo vacío: {file_name}")
                            continue
                        df_list.append(df)
                        file_list.append(file_name)
                    except Exception as e:
                        print(f"Error al leer {file_name}: {e}")
                else:
                    print(f"Archivo no encontrado: {file_name}")
    return df_list, file_list

# Define la carpeta base donde se encuentran los pacientes
base_folder = "./old adults (56+ years old)"
output_folder = "./EDA/STD"

# Verifica que la carpeta EDA existe, si no la crea
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Lista para almacenar los resultados de todos los pacientes
all_results = []

# Obtener los IDs de los pacientes de la carpeta base (solo IDs únicos)
patient_ids = sorted(set(folder for folder in os.listdir(base_folder) if folder.startswith("S") and len(folder) == 4))

# Verifica que se obtuvieron los IDs de los pacientes correctamente
print(f"Lista de pacientes: {patient_ids}")

for patient_id in patient_ids:
    patient_folder = os.path.join(base_folder, patient_id)

    # Verificar si ya se ha procesado este paciente
    output_file = os.path.join(output_folder, f"spatiotemporal_G3_{patient_id}.csv")
    if os.path.exists(output_file):
        print(f"El paciente {patient_id} ya ha sido procesado. Se omite.")
        continue  # Omite este paciente y pasa al siguiente

    # Cargar todos los DataFrames y nombres de archivo para este paciente
    df_list, file_list = load_patient_data(patient_folder, patient_id)

    # Lista para almacenar los resultados de cada toma
    results = []

    for i, df in enumerate(df_list):
        trial_name = file_list[i]
        print(f"\nProcesando la toma: {trial_name}")
        
        # Paso 1: Detección de eventos de marcha
        try:
            print("  Ejecutando la detección de gait events...")
            heel_strike_R, heel_strike_L, toe_off_R, toe_off_L = gait_events_HC_JA(df)
            print(f"  Detección de gait events exitosa para {trial_name}:")
        except Exception as e:
            print(f"  Error en la detección de gait events para {trial_name}: {e}")
            continue  # Salta esta toma si falla la detección de eventos

        # Verifica si se detectaron suficientes eventos
        if not heel_strike_R or not toe_off_R:
            print(f"  Se omite la toma {trial_name} por eventos insuficientes.")
            continue

        # Paso 2: Cálculo de variables espaciotemporales
        try:
            print("  Ejecutando el cálculo de variables espaciotemporales...")
            spatiotemporal_df, mean_df, std_df = compute_spatiotemporal_variables(df, heel_strike_R, heel_strike_L, toe_off_R, toe_off_L, sampling_rate)
            if std_df.empty:
                print(f"  Se omite la toma {trial_name} por resultados espaciotemporales vacíos.")
                continue
            print(f"  Cálculo de variables espaciotemporales completado para {trial_name}.")
        except Exception as e:
            print(f"  Error en el cálculo espaciotemporal para {trial_name}: {e}")
            continue

        # Agrega una columna para identificar la toma
        std_df['Trial'] = os.path.basename(trial_name)
        results.append(std_df)

    # Combina los resultados de todas las tomas en un solo DataFrame
    if results:
        final_df = pd.concat(results, ignore_index=True)
        print("\nVista previa del DataFrame final:")
        print(final_df.head())

        # Ruta del archivo de salida en la carpeta EDA
        output_file = os.path.join(output_folder, f"spatiotemporal_G3_{patient_id}.csv")

        # Elimina el archivo existente si ya existe
        if os.path.exists(output_file):
            os.remove(output_file)
            print(f"Archivo existente eliminado: {output_file}")

        # Guardar el DataFrame final en un archivo CSV con el patient_id en la carpeta EDA
        final_df.to_csv(output_file, index=False)
    else:
        print(f"No se generaron resultados para el paciente {patient_id}.")

Lista de pacientes: ['S009', 'S013', 'S019', 'S040', 'S081', 'S102', 'S103', 'S104', 'S107', 'S109', 'S110', 'S112', 'S113', 'S116', 'S119', 'S120', 'S123', 'S125', 'S126', 'S127', 'S128', 'S129', 'S130', 'S138', 'S139', 'S140', 'S141', 'S142', 'S143', 'S145', 'S146', 'S154', 'S156', 'S157', 'S158']

Procesando la toma: ./old adults (56+ years old)/S009/S009_G03_D01_B01_T01.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./old adults (56+ years old)/S009/S009_G03_D01_B01_T01.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./old adults (56+ years old)/S009/S009_G03_D01_B01_T01.csv.

Procesando la toma: ./old adults (56+ years old)/S009/S009_G03_D01_B01_T02.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./old adults (56+ years old)/S009/S009_G03_D01_B01_T02.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de varia

/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:212: RuntimeWarning: invalid value encountered in sqrt
  area = np.sqrt(s * (s - dist_L_to_R) * (s - dist_nextL_to_R) * (s - c))


  Cálculo de variables espaciotemporales completado para ./old adults (56+ years old)/S110/S110_G03_D01_B01_T02.csv.

Procesando la toma: ./old adults (56+ years old)/S110/S110_G03_D01_B01_T03.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./old adults (56+ years old)/S110/S110_G03_D01_B01_T03.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./old adults (56+ years old)/S110/S110_G03_D01_B01_T03.csv.

Procesando la toma: ./old adults (56+ years old)/S110/S110_G03_D01_B02_T01.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./old adults (56+ years old)/S110/S110_G03_D01_B02_T01.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./old adults (56+ years old)/S110/S110_G03_D01_B02_T01.csv.

Procesando la toma: ./old adults (56+ years old)/S110/S110_G03_D01_B02_T02.csv
  Eje

/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:383: RuntimeWarning: Mean of empty slice
  "Double Support Time (s)": np.nanmean(double_support_time),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:384: RuntimeWarning: Mean of empty slice
  "Single Support Time (s)": np.nanmean(single_support_time),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:385: RuntimeWarning: Mean of empty slice
  "Percentage Double Support (%)": np.nanmean(pct_double),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:386: RuntimeWarning: Mean of empty slice
  "Percentage Single Support (%)": np.nanmean(pct_single),
/home/dmartinez/miniconda3/envs/inv_Di/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


  Cálculo de variables espaciotemporales completado para ./old adults (56+ years old)/S126/S126_G03_D01_B02_T03.csv.

Procesando la toma: ./old adults (56+ years old)/S126/S126_G03_D01_B03_T01.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./old adults (56+ years old)/S126/S126_G03_D01_B03_T01.csv:
  Ejecutando el cálculo de variables espaciotemporales...


/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:383: RuntimeWarning: Mean of empty slice
  "Double Support Time (s)": np.nanmean(double_support_time),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:384: RuntimeWarning: Mean of empty slice
  "Single Support Time (s)": np.nanmean(single_support_time),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:385: RuntimeWarning: Mean of empty slice
  "Percentage Double Support (%)": np.nanmean(pct_double),
/home/dmartinez/Documentos/Di_Inv/SpatioTemporal_calculationv2.py:386: RuntimeWarning: Mean of empty slice
  "Percentage Single Support (%)": np.nanmean(pct_single),
/home/dmartinez/miniconda3/envs/inv_Di/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


  Cálculo de variables espaciotemporales completado para ./old adults (56+ years old)/S126/S126_G03_D01_B03_T01.csv.

Procesando la toma: ./old adults (56+ years old)/S126/S126_G03_D01_B03_T02.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./old adults (56+ years old)/S126/S126_G03_D01_B03_T02.csv:
  Ejecutando el cálculo de variables espaciotemporales...
  Cálculo de variables espaciotemporales completado para ./old adults (56+ years old)/S126/S126_G03_D01_B03_T02.csv.

Procesando la toma: ./old adults (56+ years old)/S126/S126_G03_D01_B03_T03.csv
  Ejecutando la detección de gait events...
  Detección de gait events exitosa para ./old adults (56+ years old)/S126/S126_G03_D01_B03_T03.csv:
  Se omite la toma ./old adults (56+ years old)/S126/S126_G03_D01_B03_T03.csv por eventos insuficientes.

Vista previa del DataFrame final:
   Right Step Width (cm)  Left Step Width (cm)  Right Stride Length (cm)  \
0              16.252320             13.7648

In [1]:
import os

In [5]:
parent_dir = os.getcwd()
path = os.path.join(parent_dir, 'EDA', 'Mean')
path

'/home/dmartinez/Documentos/Di_Inv/EDA/Mean'

In [7]:
!pip install PyYAML

In [23]:
# create a random list of numbers
import random
import numpy as np

random_numbers = np.random.rand(10)  # 1000 random numbers between 0 and 1
# create a random list of numbers

In [27]:
thisdict = {
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}

In [30]:
thisdict["random_numbers"] = random_numbers.tolist()
thisdict["path"] = {'path': random_numbers.tolist(), 'path2': random_numbers.tolist()}

In [10]:
import yaml


In [14]:
# create a method to save the dictionary to a yaml file
def save_dict_to_yaml(dictionary, filename):
    with open(filename, 'w') as file:
        yaml.dump(dictionary, file)

# create a method to load the dictionary from a yaml file
def load_dict_from_yaml(filename):
    with open(filename, 'r') as file:
        return yaml.safe_load(file)
# save the dictionary to a yaml file

In [31]:
save_dict_to_yaml(thisdict, 'my_dict.txt')

In [33]:
dic= load_dict_from_yaml('my_dict.txt')['path']

In [35]:
# create a dataframe from the dictionary
import pandas as pd
df = pd.DataFrame(dic)
df

,path,path2
0,0.996493,0.996493
1,0.366893,0.366893
2,0.559770,0.559770
3,0.144130,0.144130
4,0.796226,0.796226
5,0.547938,0.547938
6,0.300767,0.300767
7,0.283272,0.283272
8,0.990299,0.990299
9,0.248153,0.248153


In [36]:
# create a method that help me to print the information of the dataframe into latex table
def print_latex_table(df):
    """
    Print the dataframe as a latex table
    """
    print(df.to_latex(index=False))

In [37]:
print_latex_table(df)

\begin{tabular}{rr}
\toprule
path & path2 \\
\midrule
0.996493 & 0.996493 \\
0.366893 & 0.366893 \\
0.559770 & 0.559770 \\
0.144130 & 0.144130 \\
0.796226 & 0.796226 \\
0.547938 & 0.547938 \\
0.300767 & 0.300767 \\
0.283272 & 0.283272 \\
0.990299 & 0.990299 \\
0.248153 & 0.248153 \\
\bottomrule
\end{tabular}

